In [ ]:
import csv 
import math
import numpy

print('starting to train..')

# label_path = '/Users/apple/Desktop/workspace/20news-bydate/matlab/dgm.label'
# data_path = '/Users/apple/Desktop/workspace/20news-bydate/matlab/dgm.data'
# number_of_class=2
# number_of_words=8

label_path = '/Users/apple/Desktop/workspace/20news-bydate/matlab/train.label'
data_path = '/Users/apple/Desktop/workspace/20news-bydate/matlab/train.data'
number_of_class=20;
number_of_words=61188

# construct label dict
f = open(label_path)
reader = csv.reader(f,delimiter=" ")
label={}
i=0;
for rows in reader:
    label[i]=int(rows[0])-1
    i+=1
    
num_of_doc=len(label)

# Nk - number of doc per class
Nk=[0]*number_of_class;
for i in label:
    Nk[label[i]]=Nk[label[i]]+1;

PrC=[0]*number_of_class
for c in range(0,number_of_class):
#     PrC[c]=math.log(Nk[c])-math.log(num_of_doc)
    PrC[c]=Nk[c]/num_of_doc


min_count=1
# #twpc total nuber of words per class
twpc = [0] * number_of_class
# #fwpc freq words per class
fwpc = [[min_count] * number_of_words for i in range(number_of_class)] 
# calculate X matix d*w
X=numpy.zeros(shape=(num_of_doc,number_of_words))

f = open(data_path)
reader = csv.reader(f,delimiter=" ")
for rows in reader:
    twpc[label[int(rows[0])-1]]+=int(rows[2])
    fwpc[label[int(rows[0])-1]][int(rows[1])-1]+=int(rows[2])
    X[int(rows[0])-1][int(rows[1])-1]=int(rows[2])

# calcuate mean
mean=numpy.zeros(shape=(number_of_class,number_of_words))
for c in range(0,number_of_class):
    for w in range(0,number_of_words):
        mean[c][w]=fwpc[c][w]/Nk[c]
print("calculated mean")

# calculate covariance matrix        
sigma=numpy.zeros(shape=(number_of_words,number_of_words))
for d in range(0,num_of_doc):
    temp=X[d]-mean[label[d]]
    sigma=sigma+numpy.outer(temp,temp)

sigma=(1/num_of_doc)*sigma
print("calculated sigma")

sigma_inv=numpy.linalg.pinv(sigma)
print("calculated sigma_inv")

beta=[0]*number_of_class
# beta calcuation
for c in range(0,number_of_class):
    beta[c]=0.5*(mean[c].T.dot(sigma_inv).dot(mean[c]))+math.log(PrC[c])
print("calculated beta")

# smooth=15# smothing value

# PrWgivenC = [[0.0] * number_of_words for i in range(number_of_class)]  
# for c in range(0,number_of_class):
#     for w in range(0,number_of_words):
# #         PrWgivenC[c][w]=math.log(nk[c][w])-math.log(Nk[c])+10
#         PrWgivenC[c][w]=math.log(fwpc[c][w])-math.log(twpc[c])+smooth
        


def getClass(x):
    global beta,mean,sigma_inv
    
#     select arg max
    max_c=0
    max_val=mean[0].T.dot(sigma_inv).dot(x)
    for c in range(1,number_of_class):
        t=mean[0].T.dot(sigma_inv).dot(x)
        if max_val<t:
            max_val=t
            max_c=c
    return max_c;

# wc=numpy.array([0,0,2,0,2,3,0,0])
# wc=numpy.array([1,0,1,1,0,0,0,0])
# wc=numpy.array([0,0,0,2,0,0,3,1])
# print(getClass(wc))

print("done!")


starting to train..
calculated mean


In [ ]:
# print('starting to buid wc for test')
# test_label_path = '/Users/apple/Desktop/workspace/20news-bydate/matlab/test.label'
# test_data_path = '/Users/apple/Desktop/workspace/20news-bydate/matlab/test.data'
# f = open(test_label_path)
# reader = csv.reader(f,delimiter=" ")
# tlabel={}
# i=0;
# for rows in reader:
#     tlabel[i]=int(rows[0])-1
#     i+=1

# no_of_test_doc=7505

# wc = [[0] * number_of_words for i in range(no_of_test_doc)] 

# f = open(test_data_path)
# reader = csv.reader(f,delimiter=" ")
# for rows in reader:
#     wc[int(rows[0])-1][int(rows[1])-1]+=int(rows[2])

# print("created wc")

In [ ]:
print("starting to test")
f = open(test_label_path)
reader = csv.reader(f,delimiter=" ")
tlabel={}
i=0;
for rows in reader:
    tlabel[i]=int(rows[0])-1
    i+=1
    
correct=0
incorrect=0
for i in range(0,no_of_test_doc):
    if(i%1000==0):
        print("-- doc index:"+str(i))
    l=getClass(wc[i])
    if l==tlabel[i]:
        correct+=1
    else:
        incorrect+=1

print("final result: c: "+str(correct)+"  e: "+str(incorrect))
print("Accuracy:"+str(correct/(correct+incorrect)))

In [65]:
beta

[4.243294432868239, 0.9402139042375144]